In [0]:
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input,Dense, Dropout, Flatten
from keras.optimizers import SGD
from keras.layers import Conv2D, MaxPooling2D

import cv2
import h5py as h5py
import numpy as np
from keras.utils import np_utils
from PIL import Image

In [0]:
isshape=112
#include_top=False 迁移除顶层以外的其他网络结构。
# VGG模型对于输入图像数据要求高宽至少48个像素点。由于性能问题，我们选用48个像素点而不是VGG16的224个像素点。
# 即使这样依然需要24GB以上的内存，或者使用数据生成器
model_vgg = VGG16(include_top=False, weights = 'imagenet', input_shape=(isshape,isshape,3))

In [0]:
# 权重冻结，不允许对已有权重进行训练
for layer in model_vgg.layers:
    layer.trainable = False

In [0]:
model = Flatten(name='flatten')(model_vgg.output)
model = Dense(4096, activation='relu', name='fc1')(model)
model = Dense(4096, activation='relu', name='fc2')(model)
model = Dropout(0.5)(model)
model = Dense(10, activation = 'softmax',name='prediction')(model)
model_vgg_mnist_pretrain = Model(model_vgg.input, model, name='vgg16_pretrain')
model_vgg_mnist_pretrain.summary()

In [0]:
sgd = SGD(lr = 0.05, decay= 1e-5)
model_vgg_mnist_pretrain.compile( loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

In [0]:
#为何必须是三维的？？2维不行吗？
import os
def loaddataMnist():
  os.chdir("drive/My Drive/Colab Notebooks") 
  f = np.load('./mnist.npz')
  x_train, y_train = f['x_train'], f['y_train']
  x_test, y_test = f['x_test'], f['y_test']
  f.close()
  #将y修改为one-hot编码
  y_train = np_utils.to_categorical(y_train,10)
  y_test = np_utils.to_categorical(y_test,10)
  return (x_train,y_train),(x_test,y_test) 

In [0]:
!pwd


In [0]:
!ls "drive/My Drive/Colab Notebooks"

In [0]:
# 数据，分为训练集和测试集
(x_train, y_train), (x_test, y_test) = loaddataMnist()
y_train.shape

In [0]:
x_train = [cv2.cvtColor(cv2.resize(i, (isshape,isshape)), cv2.COLOR_GRAY2BGR) for i in x_train ]
x_train = np.concatenate([arr[np.newaxis] for arr in x_train]).astype('float32')
x_test = [cv2.cvtColor(cv2.resize(i, (isshape,isshape)), cv2.COLOR_GRAY2BGR) for i in x_test ]
x_test = np.concatenate([arr[np.newaxis] for arr in x_test]).astype('float32')

In [0]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [0]:
x_train = x_train / 255
x_test = x_test /255

In [0]:
model_vgg_mnist_pretrain.fit(x_train, y_train, validation_data = ( x_test,y_test), epochs=20, batch_size=256)

In [0]:
score=model_vgg_mnist_pretrain.evaluate(x_test,y_test)
print("test score",score[0])
print("test accuracy",score[1])